In [115]:
import os
import cv2
import json
import tqdm
import shutil
import pandas as pd

root_path = r"E:\\pgu_safety_dataset\\"
label_path = root_path+r"org_labels\\"
label_mod_path = root_path+r"mod_labels\\"
img_path = root_path+r"org_images\\"
img_mod_path = root_path+r"mod_images\\"
training_dataset_path = root_path+r"training_dataset\\"
validation_dataset_path = root_path+r"validation_dataset\\"
label_file_list = os.listdir(label_path)

1. 인창원 데이터셋의 이미지와 바운딩 박스 확인

In [34]:
org_label = json.loads(open(label_path+"label.json", "r").readlines()[0])
for idx, label in enumerate(org_label[800:820]):
    img = cv2.imread(img_path + label["file_name"])
    for bbox in label["label"]:
        x1 = int(bbox["coord"]["x"] * img.shape[1])
        y1 = int(bbox["coord"]["y"] * img.shape[0])
        x2 = int(bbox["coord"]["x"] * img.shape[1]) + int(bbox["coord"]["width"] * img.shape[0] *2)
        y2 = int(bbox["coord"]["y"] * img.shape[0]) + int(bbox["coord"]["height"] * img.shape[0])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, bbox["label_name"], (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0))
    cv2.imshow("img", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

2. 라벨링 txt 파일 생성

In [117]:
label_made = []
class_number_name = ["no helmet", "helmet"]
class_info = []
for idx, class_name in enumerate(class_number_name):
    class_info.append({
        "class_no" : str(idx),
        "class_name" : class_name,
        "image_cnt" : 0
    })

for idx, label in enumerate(org_label):
    print("{} : 라벨파일 생성중...({:.2f}%)   ".format(label["file_name"], ((idx+1)/len(org_label))*100), end="\r")
    img = cv2.imread(img_path + label["file_name"])
    w = img.shape[1]
    h = img.shape[0]
    label_str = ""
    for bbox in label["label"]:
        if bbox["label_name"] != "":
            bb_center_x = bbox["coord"]["center_x"]
            bb_center_y = bbox["coord"]["center_y"]
            bb_w = bbox["coord"]["width"]
            bb_h = bbox["coord"]["height"]
            new_class_no = [x["class_no"] for x in class_info if x["class_name"] == bbox["label_name"]][0]
            class_info_idx = [idx for idx, x in enumerate(class_info) if x["class_name"] == bbox["label_name"]][0]
            label_str += "{}\t{}\t{}\t{}\t{}\n".format(new_class_no, bb_center_x, bb_center_y, bb_w, bb_h)
            class_info[class_info_idx]["image_cnt"] = class_info[class_info_idx]["image_cnt"] + 1
    if label_str != "":
        label_made.append(label["file_name"])
        label_str = label_str[:-2]
        open(label_mod_path + label["file_name"].split(".")[0] + ".txt", "w").write(label_str)
        shutil.copyfile(img_path+"\\"+label["file_name"].split(".")[0] + ".png", img_mod_path+label["file_name"].split(".")[0] + ".png")

class_info

[{'class_no': '0', 'class_name': 'no helmet', 'image_cnt': 1671},
 {'class_no': '1', 'class_name': 'helmet', 'image_cnt': 667}]

3. 헬멧 데이터 갯수가 1천개 정도 부족. aihub 데이터셋에서 메꿔줌

In [118]:
ai_label_folder = r"E:\aihub_safety_dataset\Training_unziped\label_factory_org"
ai_image_folder = r"E:\aihub_safety_dataset\Training_unziped\image_factory"

ai_label_list = os.listdir(ai_label_folder)
cnt = 0
for label in ai_label_list:
    if cnt >= 1000:
        break

    json_data = json.loads("".join(open(ai_label_folder + "\\" + label, "r", encoding="utf-8").readlines()))
    w = json_data["image"]["resolution"][0]
    h = json_data["image"]["resolution"][1]
    bbox_infos = [x for x in json_data["annotations"] if x["class"] == "07"]
    bbox_cnt = 0
    if len(bbox_infos) > 0:
        print("{} : 라벨파일 생성중...({:.2f}%)   ".format(label, (cnt/1000)*100), end="\r")
        label_str = ""
        for bbox in bbox_infos:
            if "box" in bbox:
                bbox_cnt += 1
                bb_center_x = bbox["box"][0] + ((bbox["box"][2] - bbox["box"][0])/2)
                bb_center_y = bbox["box"][1] + ((bbox["box"][3] - bbox["box"][1])/2)
                bb_w = bbox["box"][2] - bbox["box"][0]
                bb_h = bbox["box"][3] - bbox["box"][1]
                bb_w_scaled = bb_w/w
                bb_h_scaled = bb_h/h
                bb_center_x_scaled = bb_center_x/w
                bb_center_y_scaled = bb_center_y/h
                label_str += "{}\t{}\t{}\t{}\t{}\n".format("1", bb_center_x_scaled, bb_center_y_scaled, bb_w_scaled, bb_h_scaled)
        if label_str != "":
            label_made.append(json_data["image"]["filename"])
            label_str = label_str[:-2]
            if os.path.isfile(ai_image_folder+"\\"+json_data["image"]["filename"]):
                shutil.copyfile(ai_image_folder+"\\"+json_data["image"]["filename"], img_mod_path+json_data["image"]["filename"])
                open(label_mod_path + json_data["image"]["filename"].split(".")[0] + ".txt", "w").write(label_str)
                class_info[1]["image_cnt"] = class_info[1]["image_cnt"] + bbox_cnt
                cnt += bbox_cnt

df_class_info = pd.DataFrame(class_info)
df_label_made = pd.DataFrame({ "image_name" : label_made})

df_class_info.to_excel("df_class_info.xlsx", index=False)
df_label_made.to_excel("df_label_made.xlsx", index=False)

4. 바운딩박스가 제대로 입력되었는지 확인

In [121]:
import random

label_list = os.listdir(label_mod_path)
for i in range(20):
    rand_no = random.randrange(0, len(label_list)-1)
    label = open(label_mod_path+label_list[rand_no], "r").readlines()
    img = cv2.imread(img_mod_path+label_list[rand_no].split(".")[0]+".png")
    if img is None:
        img = cv2.imread(img_mod_path+label_list[rand_no].split(".")[0]+".jpg")
    # print(label_list[rand_no])
    for bbox_str in label:
        bbox = bbox_str.split("\t")
        x1 = int((float(bbox[1]) - (float(bbox[3])/2)) * img.shape[1])
        y1 = int((float(bbox[2]) - (float(bbox[4])/2)) * img.shape[0])
        x2 = int((float(bbox[1]) + (float(bbox[3])/2)) * img.shape[1])
        y2 = int((float(bbox[2]) + (float(bbox[4])/2)) * img.shape[0])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, bbox[0], (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0))
    cv2.imshow("img", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

5. 데이터셋 분할(test, validataion)

In [122]:
df_class_info = pd.read_excel("df_class_info.xlsx")
df_label_made = pd.read_excel("df_label_made.xlsx")

In [123]:
df_class_info

,class_no,class_name,image_cnt
0,0,no helmet,1671
1,1,helmet,1668


In [124]:
df_label_made

,image_name
0,00_10.png
1,00_100.png
2,00_101.png
3,00_102.png
4,00_103.png
...,...
1974,S2-N1104M02104.jpg
1975,S2-N1104M02105.jpg
1976,S2-N1104M02106.jpg
1977,S2-N1104M02110.jpg


In [125]:
train = df_label_made.sample(frac=0.8, random_state=8699)
test = df_label_made.drop(train.index)
train

,image_name
183,00_329.png
1398,S2-N1104M00555.jpg
109,00_223.png
270,00_48.png
103,00_217.png
...,...
1245,30.png
865,07_29.png
986,07_75.png
1429,S2-N1104M00609.jpg


In [129]:
print("트레이닝 데이터셋 복사중...")
for i in tqdm.trange(len(train)):
    this_file_name = train.iloc[i]["image_name"]
    src = cv2.imread("{}{}".format(img_mod_path, this_file_name))
    resized = cv2.resize(src, (640, 640), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite("{}/images/{}".format(training_dataset_path, this_file_name), resized)
    shutil.copyfile("{}{}.txt".format(label_mod_path, this_file_name.split(".")[0]), "{}/labels/{}.txt".format(training_dataset_path, this_file_name.split(".")[0]))

print("검증 데이터셋 복사중...")
for i in tqdm.trange(len(test)):
    this_file_name = test.iloc[i]["image_name"]
    src = cv2.imread("{}{}".format(img_mod_path, this_file_name))
    resized = cv2.resize(src, (640, 640), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite("{}/images/{}".format(validation_dataset_path, this_file_name), resized)
    shutil.copyfile("{}{}.txt".format(label_mod_path, this_file_name.split(".")[0]), "{}/labels/{}.txt".format(validation_dataset_path, this_file_name.split(".")[0]))

트레이닝 데이터셋 복사중...


100%|██████████| 1583/1583 [02:11<00:00, 12.04it/s]


검증 데이터셋 복사중...


100%|██████████| 396/396 [00:33<00:00, 11.93it/s]


아래 명령어로 트레이닝 실행!

python C:\Users\poscohrd\Desktop\yolo_transfer_learning\yolov5-master\train.py --img 640 --batch 16 --epochs 30 --data C:\Users\poscohrd\Desktop\yolo_transfer_learning\yolov5-master\dataset.yaml --weights yolov5m.pt --device 0,1

*배치 사이즈가 너무 클 경우 GPU 메모리 에러 발생함 주의할 것

학습이 끝나면 아래와 같이 tensorflow 모델로 변환(필요시)

python yolov5-master\export.py --weights yolov5-master\runs\train\exp11\weights\best.pt --include pb